<a href="https://colab.research.google.com/github/ekingit/DeepForecast/blob/main/1_1_Parameter_opt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader

from google.colab import drive
drive.mount('/content/gdrive')

!cp /content/gdrive/MyDrive/Colab\ Notebooks/weather_forecast/Data/data.py /content/data.py
!cp /content/gdrive/MyDrive/Colab\ Notebooks/weather_forecast/models.py /content/models.py
!cp /content/gdrive/MyDrive/Colab\ Notebooks/weather_forecast/train.py /content/train.py

from data import Sine_Data, Weather_Data
from models import local_LSTM
from train import train_lstm, test_lstm

Mounted at /content/gdrive


In [ ]:
data_loc = '/content/gdrive/MyDrive/Colab Notebooks/weather_forecast/Data/weather_prediction_dataset.csv'
data_column = 'BASEL_temp_max'
nat_data = Weather_Data(data_loc, data_column)

In [ ]:
#create an empty .csv to save the results
columns = ['model', 'seq_len', 'hidden_size', 'num_layers', 'run_time', 'device','best_epoch', 'num_epoch',
           'test_loss','val_loss','train_loss']
df = pd.DataFrame(columns=columns)
res_loc = '/content/gdrive/MyDrive/Colab Notebooks/weather_forecast/Results/Hyperparameter_Performance_Analysis.csv'
df.to_csv(res_loc, index=True)

In [ ]:
def main(ind, seq_len, hidden_size, num_layers, lr=1e-4, num_epoch=1500, target_seq_len=7, batch_size=128):
  '''batch data, define model, train, test, save best model (minimum validation error),
     write the results into the .csv file, save train-val plot'''
  save_model_path =f'/content/gdrive/MyDrive/Colab Notebooks/weather_forecast/Results/model{ind}.pth'
  model_name = 'local_LSTM'
  device = 'cuda'
  #data
  (in_train, out_train), (in_val,out_val), (in_test,out_test) = nat_data.data_chunks(seq_len, target_seq_len)
  ds_train = TensorDataset(in_train, out_train)
  dl_train = DataLoader(ds_train, batch_size=batch_size, shuffle=False)
  ds_val = TensorDataset(in_val, out_val)
  dl_val = DataLoader(ds_val, batch_size=batch_size, shuffle=False)
  ds_test = TensorDataset(in_test, out_test)
  dl_test = DataLoader(ds_test, batch_size=batch_size, shuffle=False)
  #model
  model = local_LSTM(1,hidden_size,num_layers)
  optimizer = torch.optim.Adam(model.parameters(), lr=lr)
  mse_loss = nn.MSELoss(reduction='sum')
  #RUN!
  train_loss_list = []
  val_loss_list = []
  best_loss = 0.5
  a=time.time()
  for epoch in range(num_epoch):
      train_loss = train_lstm(model, dl_train, optimizer, mse_loss, hidden_size, num_layers, device=device)
      val_loss = test_lstm(model, dl_val, mse_loss, hidden_size, num_layers, device=device)
      if val_loss < best_loss:
          best_loss = val_loss
          best_epoch = epoch
          best_train = train_loss
          test_loss = test_lstm(model, dl_test, mse_loss, hidden_size, num_layers, device=device)
          torch.save(model.state_dict(), save_model_path)
      train_loss_list.append(train_loss)
      val_loss_list.append(val_loss)
      if epoch %100 == 0:
              print(f"epoch: {epoch} train loss: {train_loss}, validation loss: {val_loss}")
  b=time.time()
  #write results to csv
  results = {'model':model_name, 'seq_len':seq_len, 'hidden_size':hidden_size, 'num_layers':num_layers}
  results['run_time'] = b-a
  results['device'] = 'T4_GPU'
  results['best_epoch'] = best_epoch
  results['num_epoch'] = num_epoch
  results['test_loss'] = test_loss
  results['val_loss'] = best_loss
  results['train_loss'] = best_train

  df1 = pd.DataFrame(results, index=[ind])
  df1.to_csv(res_loc, mode='a', header=False)
  save_plot_path = f'/content/gdrive/MyDrive/Colab Notebooks/weather_forecast/Results/plt{ind}.png'
  train_loss_tens = torch.tensor(train_loss_list)
  val_loss_tens = torch.tensor(val_loss_list)
  plt.plot(train_loss_tens.numpy(),label='train')
  plt.plot(val_loss_tens.numpy(),label='test')
  plt.axvline(x=best_epoch, color='r', linestyle='--')
  plt.title(f'seq_len={seq_len},hidden_size={hidden_size},num_layers={num_layers}')
  plt.xlabel('epoch')
  plt.ylabel('MSE')
  plt.legend()
  plt.savefig(save_plot_path)
  plt.show()

In [ ]:
# Run this (random grid search) or run by hand after analyzing results in every step
ind = 0
seq_len_list = [14, 30, 50]
hidden_size = [10, 20, 50, 100, 200, 300]
num_layers = [1, 2, 3]
for seq_len in seq_len_list:
  for hidden in hidden_size:
    for num in num_layers:
      main(ind, seq_len, hidden, num)
      ind += 1

In [ ]:
# Sanity check: compare results with moving avarage.
def moving_avarage(data_in, target_seq_len):
    Z = data_in.clone()
    result = torch.zeros(data_in.shape[0],target_seq_len,1)
    result[:,0] = Z.mean(1)
    for j in range(1,target_seq_len):
        Z = torch.cat([Z[:,1:],Z.mean(1).unsqueeze(-1)],1)
        result[:,j] = Z.mean(1)
    return result

In [ ]:
#Sanity check - Moving avarage
ind = -1
seq_len = 14
target_seq_len = 7
#data
(in_train, out_train), (in_val,out_val), (in_test,out_test) = nat_data.data_chunks(seq_len, target_seq_len)
#model run
loss = nn.MSELoss()
a = time.time()

train_moving_avg_preds = moving_avarage(in_train, target_seq_len)
train_moving_avg_loss = loss(train_moving_avg_preds, out_train)

val_moving_avg_preds = moving_avarage(in_val, target_seq_len)
val_moving_avg_loss = loss(val_moving_avg_preds, out_val)

test_moving_avg_preds = moving_avarage(in_test, target_seq_len)
test_moving_avg_loss = loss(test_moving_avg_preds, out_test)

b = time.time()
#results
results = {'model':'moving_avarage', 'seq_len':14, 'hidden_size':0, 'num_layers':0}
results['run_time'] = b-a
results['device'] = 'T4_GPU'
results['best_epoch'] = 0
results['num_epoch'] = 0
results['test_loss'] = test_moving_avg_loss.item()
results['val_loss'] = val_moving_avg_loss.item()
results['train_loss'] = train_moving_avg_loss.item()
#save results
res_loc = '/content/gdrive/MyDrive/Colab Notebooks/weather_forecast/Results/Hyperparameter_Performance_Analysis.csv'
df1 = pd.DataFrame(results, index=[ind])
df1.to_csv(res_loc, mode='a', header=False)